#Project 2

- Student 1: 

Chandni Patel

A20455322

- Student 2:

Omkar Pawar

A20448802

In [35]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

import nltk, re, string, collections
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
import csv
from scipy import sparse

from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from sklearn.neural_network import MLPClassifier
import pickle

## Load Preprocessed train and test data

In [36]:
train_preprocessed = pd.read_csv("train_preprocessed.csv")
train_labels = pd.read_csv("train.csv",usecols=[5])
test_preprocessed = pd.read_csv("test_preprocessed.csv")

Create Vocabulary 

In [37]:
vectorizer = CountVectorizer()
feature = train_preprocessed["title1_en"] + train_preprocessed["title2_en"]
train_headline_counts = vectorizer.fit_transform(list(feature))
vocab = (list(vectorizer.vocabulary_.keys()))

vectorizer = CountVectorizer(vocabulary=vocab)
transformer = TfidfTransformer()

Create features using TF - IDF for training

In [24]:
#Title1
title_1 = train_preprocessed["title1_en"]

title1_counts = vectorizer.fit_transform(list(title_1))
title1_tfidf = transformer.fit_transform(title1_counts)

#Title 2
title_2 = train_preprocessed["title2_en"]

title2_counts = vectorizer.fit_transform(list(title_2))
title2_tfidf = transformer.fit_transform(title2_counts)

# Create a feature with two tfidf columns
final_feature = sparse.hstack([title1_tfidf,title2_tfidf])


# Similarity Approach

Functions to get similarities

In [5]:
def cosine(trans):
  return (cosine_similarity(trans[0],trans[1]))[0][0]

def manhattan_distance(trans):
  return (pairwise_distances(trans[0],trans[1],metric = "manhattan")[0][0])

def euclidean_distance(trans):
  euc=euclidean_distances(trans[0], trans[1])
  return euc[0][0]

Create features using similarity

In [6]:
def similarity_features(df,title1_counts,title2_counts):
  count = len(df["title1_en"])
  cos_similarities = []
  #manhattan_ = []
  #e_distances = []

  for i in (range(count)):  
    corpus = [title1_counts[i],title2_counts[i]]
    #trans = vectorizer.transform(corpus)
    cos_similarities.append(cosine(corpus))
    #manhattan_.append(manhattan_distance(trans))
    #e_distances.append(euclidean_distance(trans))

  new_train = pd.DataFrame()
  new_train["cos"] = cos_similarities
  #new_train["manh"] = manhattan_
  #new_train["euc"] = e_distances

  return new_train

In [7]:
feature_train = similarity_features(train_preprocessed,title1_counts,title2_counts)

# Data Modeling

### Naive Bayes
- Using Similarity feature

In [9]:
mnb = MultinomialNB()
mnb.fit(feature_train, train_labels)
mnb_preds = mnb.predict(feature_train)
accuracy_score(mnb_preds,train_labels)

C:\Users\imcha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.6847474282683804

- Using TF - IDF Feature

In [10]:
mnb = MultinomialNB()
mnb.fit(final_feature, train_labels)
mnb_preds = mnb.predict(final_feature)
accuracy_score(mnb_preds,train_labels)

C:\Users\imcha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.7891881985010256

### Logistic Regression
- Using Similarity feature

In [11]:
lr = LogisticRegression(solver = 'saga',multi_class='multinomial')
lr.fit(feature_train, train_labels)
lr.score(feature_train,train_labels)

C:\Users\imcha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.7502593178964444

- Using TF - IDF Feature

In [12]:
lr = LogisticRegression(solver = 'saga',multi_class='multinomial')
lr.fit(final_feature, train_labels)
lr.score(final_feature,train_labels)

C:\Users\imcha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.8548248726807621

### Multi-Layer Perceptron Classifier
- Using Similarity feature

In [13]:
mlp = MLPClassifier(activation = "relu",
                    random_state=1, 
                    max_iter=20,
                    solver = "adam",
                    learning_rate = "adaptive",
                    batch_size = 128,
                    hidden_layer_sizes = (100),
                    verbose = True,
                    shuffle = True)

mlp.fit(feature_train,train_labels)
mlp.score(feature_train,train_labels)

C:\Users\imcha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.60131442
Iteration 2, loss = 0.58693848
Iteration 3, loss = 0.58650514
Iteration 4, loss = 0.58629866
Iteration 5, loss = 0.58630287
Iteration 6, loss = 0.58628244
Iteration 7, loss = 0.58609032
Iteration 8, loss = 0.58620666
Iteration 9, loss = 0.58609599
Iteration 10, loss = 0.58607179
Iteration 11, loss = 0.58607914
Iteration 12, loss = 0.58602732
Iteration 13, loss = 0.58595573
Iteration 14, loss = 0.58604989
Iteration 15, loss = 0.58601322
Iteration 16, loss = 0.58604475
Iteration 17, loss = 0.58595376
Iteration 18, loss = 0.58602343
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


0.7501150357585731

- Using TF - IDF Feature

In [14]:
mlp = MLPClassifier(activation = "relu",
                    random_state=1, 
                    max_iter=20,
                    solver = "adam",
                    learning_rate = "adaptive",
                    batch_size = 128,
                    hidden_layer_sizes = (100),
                    verbose = True,
                    shuffle = True)

mlp.fit(final_feature,train_labels)

C:\Users\imcha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.47291546
Iteration 2, loss = 0.30305561
Iteration 3, loss = 0.21700067
Iteration 4, loss = 0.15996584
Iteration 5, loss = 0.11862220
Iteration 6, loss = 0.08926402
Iteration 7, loss = 0.06853299
Iteration 8, loss = 0.05372237
Iteration 9, loss = 0.04416687
Iteration 10, loss = 0.03710379
Iteration 11, loss = 0.03255048
Iteration 12, loss = 0.02902094
Iteration 13, loss = 0.02654663
Iteration 14, loss = 0.02527384
Iteration 15, loss = 0.02333163
Iteration 16, loss = 0.02244587
Iteration 17, loss = 0.02183874
Iteration 18, loss = 0.02106057
Iteration 19, loss = 0.02001608
Iteration 20, loss = 0.01990010


C:\Users\imcha\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(batch_size=128, hidden_layer_sizes=100, learning_rate='adaptive',
              max_iter=20, random_state=1, verbose=True)

In [29]:
mlp.score(final_feature,train_labels)

0.9982374182076259

- Save the best model

In [ ]:
pickle.dump(mlp, open("MLP_Model.sav", 'wb'))

### Testing 
Create TF - IDF features for test data

In [25]:
#Title1
test_title_1 = test_preprocessed["title1_en"]

test_title1_counts = vectorizer.fit_transform(list(test_title_1))
test_title1_tfidf = transformer.fit_transform(test_title1_counts)

#Title 2
test_title_2 = test_preprocessed["title2_en"]

test_title2_counts = vectorizer.fit_transform(list(test_title_2))
test_title2_tfidf = transformer.fit_transform(test_title2_counts)

# Create a feature with two tfidf columns
final_feature_test = sparse.hstack([test_title1_tfidf,test_title2_tfidf])

Predict and create submission for testing

In [32]:
def create_submission(test_preprocessed,pred):
  test_id = test_preprocessed["id"]
  submit = pd.DataFrame()
  submit["id"] = test_id
  submit["label"] = pred
  submit.to_csv("submission.csv",index = False)

In [33]:
test_preds = mlp.predict(final_feature_test)
create_submission(test_preprocessed,test_preds)

Load best model (if required)

In [30]:
best_model = pickle.load(open("MLP_Model.sav", 'rb'))
test_preds = best_model.predict(final_feature_test)
create_submission(test_preprocessed,test_preds)